#### In this notebook I only clean unwanted dimensions and pick desired values for each dimension.

In [1]:
import os
from dotenv import load_dotenv

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, URL

In [3]:
load_dotenv()

pg_user = os.environ["POSTGRES_USER"]
pg_db = os.environ["POSTGRES_DB"]

url = URL.create(
  "postgresql+psycopg2",
  username=pg_user,
  database=pg_db
)

con = create_engine(url)

In [ ]:
bop = pd.read_sql_table("balance_of_pay", con=con)
bop.drop("index", axis="columns", inplace=True)
bop.head()

,index,REF_AREA,COUNTERPART_AREA,MEASURE,ACCOUNTING_ENTRY,FS_ENTRY,FREQ,UNIT_MEASURE,ADJUSTMENT,TIME_PERIOD,OBS_VALUE,OBS_STATUS,UNIT_MULT,CURRENCY,DECIMALS
0,0,0,0,0,0,0,0,0,0,2003-04-01,16209.56,0,6,0,2
1,1,0,0,0,0,0,0,0,0,2003-07-01,16491.17,0,6,0,2
2,2,0,0,0,0,0,0,0,0,2003-10-01,16172.25,0,6,0,2
3,3,0,0,0,0,0,0,0,0,2004-01-01,16250.90,0,6,0,2
4,4,0,0,0,0,0,0,0,0,2004-04-01,16740.05,0,6,0,2


In [16]:
bop.min() - bop.max() == 0

REF_AREA            False
COUNTERPART_AREA     True
MEASURE             False
ACCOUNTING_ENTRY    False
FS_ENTRY             True
FREQ                False
UNIT_MEASURE        False
ADJUSTMENT          False
TIME_PERIOD         False
OBS_VALUE           False
OBS_STATUS          False
UNIT_MULT           False
CURRENCY            False
DECIMALS             True
dtype: bool

#### Firstly, the `COUNTERPART_AREA`, `FS_ENTRY` and `DECIMALS` have only one unique value. Thus they have to be deleted(both columns and dimension tables in the database).

In [ ]:
from sqlalchemy.sql import text

In [36]:
def drop_columns(tbl: str, cols: list[str], engine) -> None: 
  if len(cols) == 0:
    return
  
  query = f"ALTER TABLE {tbl}"

  for col in cols:
    query += f'\nDROP COLUMN "{col}",'

  with engine.connect() as con:
    query = query.strip(",")
    query += ";"
    con.execute(text(query))

In [ ]:
delete_bop_cols = ["COUNTERPART_AREA", "FS_ENTRY", "DECIMALS"]
drop_columns("balance_of_pay", delete_bop_cols, con)

In [39]:
rates = pd.read_sql_table("interest_rate", con)
rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1548250 entries, 0 to 1548249
Data columns (total 14 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   index           1548250 non-null  int64  
 1   REF_AREA        1548250 non-null  int64  
 2   FREQ            1548250 non-null  int64  
 3   MEASURE         1548250 non-null  int64  
 4   UNIT_MEASURE    1548250 non-null  int64  
 5   ACTIVITY        1548250 non-null  int64  
 6   ADJUSTMENT      1548250 non-null  int64  
 7   TRANSFORMATION  1548250 non-null  int64  
 8   TIME_PERIOD     1548250 non-null  int64  
 9   OBS_VALUE       1548206 non-null  float64
 10  OBS_STATUS      1548250 non-null  int64  
 11  UNIT_MULT       1548250 non-null  int64  
 12  DECIMALS        1548250 non-null  int64  
 13  BASE_PER        302090 non-null   float64
dtypes: float64(2), int64(12)
memory usage: 165.4 MB


In [40]:
rates.max() - rates.min() == 0

index             False
REF_AREA          False
FREQ              False
MEASURE           False
UNIT_MEASURE      False
ACTIVITY          False
ADJUSTMENT        False
TRANSFORMATION    False
TIME_PERIOD       False
OBS_VALUE         False
OBS_STATUS        False
UNIT_MULT         False
DECIMALS           True
BASE_PER           True
dtype: bool

### Here only `DECIMALS` and `BASE_PER` are redundant, others may make sense

In [ ]:
rates_drop_columns = ["DECIMALS", "BASE_PER"]
drop_columns("interest_rate", rates_drop_columns, con)

In [ ]:
exr = pd.read_sql_table("exchange_rates", con=con)

In [ ]:
exr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 924697 entries, 0 to 924696
Data columns (total 32 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   index            924697 non-null  int64  
 1   FREQ             924697 non-null  int64  
 2   CURRENCY         924697 non-null  int64  
 3   CURRENCY_DENOM   924697 non-null  int64  
 4   EXR_TYPE         924697 non-null  int64  
 5   EXR_SUFFIX       924697 non-null  int64  
 6   TIME_PERIOD      924697 non-null  int64  
 7   OBS_VALUE        715662 non-null  float64
 8   OBS_STATUS       924697 non-null  int64  
 9   OBS_CONF         924697 non-null  int64  
 10  OBS_PRE_BREAK    0 non-null       float64
 11  OBS_COM          924697 non-null  int64  
 12  TIME_FORMAT      924697 non-null  int64  
 13  BREAKS           0 non-null       float64
 14  COLLECTION       924697 non-null  int64  
 15  COMPILING_ORG    0 non-null       float64
 16  DISS_ORG         0 non-null       floa

#### A lot of these are NULL, so these have to be omitted.
#### Also, UNIT_MULT has only one unique value

In [57]:
mask = exr.apply(lambda x: sum(x.isna()), axis="index")

In [63]:
delete_cols = exr.keys()[mask == exr.shape[0]]
drop_columns("exchange_rates", list(delete_cols), con)

#### Now, apart from this, we need to delete the dimension tables and dinamically create foreign keys

In [75]:
drop_tbls = (
  [f"bop_{col}" for col in delete_bop_cols] +
  [f"ex_rates_{col}" for col in delete_cols] +
  [f"int_rates_{col}" for col in rates_drop_columns]
)

with con.connect() as conn:
  for tbl in drop_tbls:
    query = text(f"DROP TABLE {tbl.lower()}")

    try:
      conn.execute(query)
    except:
      print(f"Failed to delete {tbl.lower()}")
      continue

Failed to delete bop_decimals
Failed to delete ex_rates_obs_pre_break
Failed to delete ex_rates_breaks
Failed to delete ex_rates_compiling_org
Failed to delete ex_rates_diss_org
Failed to delete ex_rates_dom_ser_ids
Failed to delete ex_rates_publ_ecb
Failed to delete ex_rates_publ_mu
Failed to delete ex_rates_publ_public
Failed to delete ex_rates_coverage
Failed to delete ex_rates_nat_title
Failed to delete ex_rates_source_pub
Failed to delete int_rates_decimals
Failed to delete int_rates_base_per
